In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

### Pulling down the Listing Data

In [2]:
r = requests.get('https://www.propertypro.ng/property-for-rent/flat-apartment/?search=&bedroom=&min_price=&max_price=3000000')

soup = BeautifulSoup(r.content, 'html5lib')

In [3]:
listing_divs = soup.select('div[class=single-room-sale\ listings-property]')
print("Number of apartment listings on a page:", len(listing_divs))

Number of apartment listings on a page: 20


In [4]:
listing_divs[0]

<div class="single-room-sale listings-property">
<div class="single-room-img">
<a class="single-room-img-a" href="/property/3-bedroom-flat-apartment-for-rent-ikota-lekki-lagos-1ECXR"> <img alt="3 bedroom Flat / Apartment for rent Ikota Lekki Lagos" class="listings-img" src="https://images.propertypro.ng/medium/e7exz-premium-3-bedroom-apartment-1FN5ThN4XD5FYU2zFUZl.jpg" title="3 bedroom Flat / Apartment for rent Ikota Lekki Lagos"/> </a>
<div class="save-heart">
<a href="/profile/add-favorite/575286"><i class="fa fa-heart"></i></a>
</div>
<div class="room-icons">
<span><span>37</span> <img alt="icon" src="/assets/assets/img/property/2f8b655773948c43f77e8c928e94bbe3-img.png"/> </span>
</div>

</div>
<div class="single-room-text">
<a href="/property/3-bedroom-flat-apartment-for-rent-ikota-lekki-lagos-1ECXR"> <h2 class="listings-property-title">3 BEDROOM FLAT / APARTMENT FOR RENT</h2> </a>
<h4><img alt="icon" src="/assets/assets/img/resultpage/a6b70a0c1c4423f60780bdda189b91e3-location.svg"

In [5]:
listing_divs[0].select('h4')[0].text

'Ikota Lekki Lagos'

In [6]:
listing_divs[0].select('h3[class*=listings-price]')[0].text.strip()

'₦ 2,800,000'

In [7]:
listing_divs[0].select('div[class*=fur-areea]')[0].text.strip().split('\n')

['3 beds', '3 baths', '4 Toilets']

In [8]:
listing_divs[7].select('div[class*=furnished-btn]')[0].text.strip().replace('\n', ' ')

'Serviced Newly Built'

In [9]:
listing_divs[7].select('div[class*=result-list-details]')[0].p.text.strip().replace('Read more', '').replace('FOR RENT: ', '')

'3 bedroom Flat / Apartment for rent Old Ikoyi Ikoyi Lagos.. '

**Now that I've gotten all the relevant information, I'll put all the codes together;**

In [10]:
all_listings_data = []
for indv_listing in listing_divs:
    address = indv_listing.select('h4')[0].text
    price = indv_listing.select('h3[class*=listings-price]')[0].text.strip()
    listing_data = [address, price]
    utilities = indv_listing.select('div[class*=fur-areea]')[0].text.strip().split('\n')
    listing_data.extend(utilities)
    description = listing_divs[7].select('div[class*=result-list-details]')[0].p.text.strip().replace('Read more', '').replace('FOR RENT: ', '')
    extra = indv_listing.select('div[class*=furnished-btn]')[0].text.strip().replace('\n', ' ')
    listing_data.append(extra + description)
    all_listings_data.append(listing_data)
    
pd.DataFrame(all_listings_data)

,0,1,2,3,4,5
0,Ikota Lekki Lagos,"₦ 2,800,000",3 beds,3 baths,4 Toilets,Newly Built3 bedroom Flat / Apartment for rent...
1,OLALEYE Estate Alaka Iponri Surulere Lagos,"₦ 2,500,000",3 beds,4 baths,4 Toilets,3 bedroom Flat / Apartment for rent Old Ikoyi ...
2,"Off Kudirat Abiola way, Julie EstateOregun Ike...","₦ 2,500,000",3 beds,3 baths,4 Toilets,3 bedroom Flat / Apartment for rent Old Ikoyi ...
3,Ikate Lekki Lagos,"₦ 3,000,000",3 beds,4 baths,4 Toilets,Serviced3 bedroom Flat / Apartment for rent Ol...
4,Lekki Phase 1 Lekki Lagos,"₦ 2,800,000",3 beds,3 baths,4 Toilets,3 bedroom Flat / Apartment for rent Old Ikoyi ...
5,Lekki Phase 1 Lekki Lagos,"₦ 3,000,000",3 beds,3 baths,4 Toilets,3 bedroom Flat / Apartment for rent Old Ikoyi ...
6,ShimawaMowe Obafemi Owode Ogun,"₦ 260,000",3 beds,1 baths,2 Toilets,Newly Built3 bedroom Flat / Apartment for rent...
7,-Old Ikoyi Ikoyi Lagos,"$ 40,000",3 beds,3 baths,4 Toilets,Serviced Newly Built3 bedroom Flat / Apartment...
8,Oniru Victoria Island Lagos,"₦ 3,000,000",3 beds,3 baths,4 Toilets,Serviced3 bedroom Flat / Apartment for rent Ol...
9,Ologolo Lekki Lagos,"₦ 2,600,000",3 beds,3 baths,4 Toilets,Serviced Newly Built3 bedroom Flat / Apartment...


**Writing a dynamic function to get the data;**

In [11]:
def parse_listing_data(location, max_price, rows_of_data=2000):
    all_listings_data = []
    page_num = 0
    
    while page_num < (rows_of_data / 20):
        url = "https://www.propertypro.ng/property-for-rent/flat-apartment/?search=" + location \
              + "&bedroom=&min_price=&max_price=" + str(max_price) + "&page=" + str(page_num)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html5lib')
        listing_divs = soup.select('div[class=single-room-sale\ listings-property]')
        if len(listing_divs) == 0:
            break 
            
        for indv_listing in listing_divs:
            address = indv_listing.select('h4')[0].text
            price = indv_listing.select('h3[class*=listings-price]')[0].text.strip()
            listing_data = [address, price]
            utilities = indv_listing.select('div[class*=fur-areea]')[0].text.strip().split('\n')
            listing_data.extend(utilities)
            description = indv_listing.select('div[class*=result-list-details]')[0].p.text.strip().replace('Read more', '').replace('FOR RENT: ', '')
            extra = indv_listing.select('div[class*=furnished-btn]')[0].text.strip().replace('\n', ' ')
            listing_data.append(extra + description)
            all_listings_data.append(listing_data)
        page_num += 1    
    listing_df = pd.DataFrame(all_listings_data, columns=['neighborhood', 'price', 'beds', 'baths', 'toilets', 'extra'])
    
    listing_df.to_csv("rent_listings_"+location+".csv", index=False)
    return listing_df

### Getting Lagos' Listing Data

In [12]:
lagos_data = parse_listing_data('lagos', 2500000, 5000)
lagos_data

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

### Getting Ibadan's Data

In [ ]:
ibadan_data = parse_listing_data('oyo', 3000000, 2000)
ibadan_data

### Getting Abuja's Data

In [ ]:
abuja_data = parse_listing_data('abuja', 4000000, 3000)
abuja_data

### Getting Ogun's Data

In [ ]:
ogun_data = parse_listing_data('ogun', 2000000, 2000)
ogun_data

### Getting PortHarcourt's Data

In [ ]:
ph_data = parse_listing_data('rivers', 3000000, 2500)
ph_data